# [Puntajes PSU](http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016)

In [1]:
from robobrowser import RoboBrowser
import geocoder
import folium
import pandas
import os
from utils import persist_to_file
cwd = os.getcwd()

browser = RoboBrowser(history=True, parser='html5lib')
url = 'http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016'
browser.open(url)

In [2]:
table=browser.select('.article-content table')[0]
t=table.__str__()
t[:1000]

'<table border="0" cellpadding="0" cellspacing="0" style="width:1059px;" width="1058">\n\t<colgroup>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t</colgroup>\n\t<tbody>\n\t\t<tr height="20">\n\t\t\t<td height="20" style="height: 20px; width: 143px; text-align: center;"><strong>Posición en el ranking</strong></td>\n\t\t\t<td style="width: 159px; text-align: center;"><strong>Tipo de establecimiento</strong></td>\n\t\t\t<td style="width: 131px; text-align: center;"><strong>Comuna\xa0</strong></td>\n\t\t\t<td style="width: 332px; text-align: center;"><strong>Colegio</strong></td>\n\t\t\t<td style="width: 215px; text-align: center;"><strong>Alumnos que rindieron la prueba</strong></td>\n\t\t\t<td style="width: 80px; text-align: center;"><strong>Promedio PSU</strong></td>\n\t\t</tr>\n\t\t<tr height="20">\n\t\t\t<td height="20" style="height: 20px; text-align: center;">1</td>\n\t\t\t<td style="text-align: center;">Particular</td>\n\t\t\t<td style="text-align: cen

In [3]:
t=t.replace(',','.')
df=pandas.read_html(t, header=0)[0]
df.head(10)

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU
0,1,Particular,Providencia,Cambridge College,29,704.4
1,2,Particular,Las Condes,Cordillera,63,699.0
2,3,Particular,Concón,Montemar,22,694.0
3,4,Particular,Maipú,Internacional Alba,26,693.4
4,5,Particular,Vitacura,Los Andes,69,692.9
5,6,Particular,Puerto Montt,Instituto Alemán,46,690.9
6,7,Particular,Lo Barnechea,Maimonides School,10,689.7
7,8,Particular,Chiguayante,Pinares,16,685.5
8,9,Particular,Angol,San José,19,683.7
9,10,Particular,La Reina,The Grange School,133,682.6


In [4]:
g=geocoder.google('colegio the grange school, la reina, chile')
print(g)
print(g.latlng)

<[OK] Google - Geocode #2 results>
[-33.438182, -70.5685989]


In [5]:
g.latlng

[-33.438182, -70.5685989]

[-36.8883627, -73.0351594](
https://www.google.cl/maps/@-33.4382165,-70.5686165,3a,75y,3.98h,101.74t/data=!3m7!1e1!3m5!1sxP13kKF1tvm6pau-DNnsPA!2e0!6s%2F%2Fgeo1.ggpht.com%2Fcbk%3Fpanoid%3DxP13kKF1tvm6pau-DNnsPA%26output%3Dthumbnail%26cb_client%3Dmaps_sv.tactile.gps%26thumb%3D2%26w%3D203%26h%3D100%26yaw%3D33.012394%26pitch%3D0%26thumbfov%3D100!7i13312!8i6656)

In [6]:
import time

@persist_to_file()
def get_latlng(query):
    providers = geocoder.osm, geocoder.google
    for provider in providers:
        #print(provider)
        try:  
            g=provider(query)
        except ValueError:
            continue
        if g:
            return g.latlng
    return None, None

def get_latlng_nocache(query):
    providers = geocoder.osm, geocoder.google
    for provider in providers:
        #print(provider)
        try:  
            g=provider(query)
        except ValueError:
            continue
        if g:
            return g.latlng
    return None, None

for index,row in df.iterrows():
    stopwords = 'colegio', 'liceo', 'school', 'instituto', 'college'
    query = row.Colegio +", "+ row.Comuna + ", chile"
    query = query.lower()
    for i in stopwords:
        if i in query:
            break
    else:
         query = 'colegio ' + query
    print(query),
    g=get_latlng(query)
    time.sleep(0.1)
    if not g:
        print('ERROR', query)
        continue
    if g:
        lat, lng = g
        df.set_value(index,'lat', lat)
        df.set_value(index,'lng', lng)
        print(g)
    else:
        print(g, 'ERROR', query)

cambridge college, providencia, chile
[-33.4297301, -70.6171096621219]
colegio cordillera, las condes, chile
[-33.39712965, -70.5143209449632]
colegio montemar, concón, chile
[-32.95104785, -71.5354626703725]
colegio internacional alba, maipú, chile
[-33.4805743, -70.74499]
colegio los andes, vitacura, chile
[-33.37579495, -70.5270086976089]
instituto alemán, puerto montt, chile
[-41.4703149, -72.9327801328877]
maimonides school, lo barnechea, chile
[-33.362362, -70.485016]
colegio pinares, chiguayante, chile
[-36.9329182, -73.0240795401104]
colegio san josé, angol, chile
[None, None]
the grange school, la reina, chile
[-33.43693825, -70.5678364497786]
instituto hebreo, lo barnechea, chile
[-33.36782585, -70.50897645]
colegio trebulco, talagante, chile
[-33.68492175, -70.9181438684788]
colegio tabancura, vitacura, chile
[-33.3803347, -70.5348377]
the kent school, providencia, chile
[-33.442454, -70.606202]
colegio la girouette, las condes, chile
[-33.42014435, -70.5684437802117]
colegi

In [7]:
df[df.isnull().any(axis=1)]

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
8,9,Particular,Angol,San José,19,683.7,NaN,NaN
22,23,Particular,Talca,Particular Montessori,53,669.8,NaN,NaN
27,28,Particular,Las Condes,The Southern Cross School,31,668.6,NaN,NaN
44,45,Particular,Curicó,Alianza Francesa J. Mermoz,28,660.1,NaN,NaN
48,49,Particular,San Pedro de la Paz,Alemán de Concepción,68,658.4,NaN,NaN
76,77,Particular,La Serena,Scuola Italiana Alcide De Gasperi,45,649.7,NaN,NaN
78,79,Particular,Rancuagua,Instituto Inglés Rancagua,76,649.2,NaN,NaN
81,82,Particular,La Reina,Colegio British Royal School,63,648.0,NaN,NaN
93,94,Particular,Calama,Colegio San Ignacio,27,643.4,NaN,NaN
96,97,Subvencionado,Copiapó,Colegio San Francisco de la Selva,15,643.2,NaN,NaN


In [8]:
df=df.dropna()
df.head(25)

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
0,1,Particular,Providencia,Cambridge College,29,704.4,-33.429730,-70.617110
1,2,Particular,Las Condes,Cordillera,63,699.0,-33.397130,-70.514321
2,3,Particular,Concón,Montemar,22,694.0,-32.951048,-71.535463
3,4,Particular,Maipú,Internacional Alba,26,693.4,-33.480574,-70.744990
4,5,Particular,Vitacura,Los Andes,69,692.9,-33.375795,-70.527009
5,6,Particular,Puerto Montt,Instituto Alemán,46,690.9,-41.470315,-72.932780
6,7,Particular,Lo Barnechea,Maimonides School,10,689.7,-33.362362,-70.485016
7,8,Particular,Chiguayante,Pinares,16,685.5,-36.932918,-73.024080
9,10,Particular,La Reina,The Grange School,133,682.6,-33.436938,-70.567836
10,11,Particular,Lo Barnechea,Instituto Hebreo,76,681.0,-33.367826,-70.508976


In [9]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Terrain')
i=0
for index, row in df.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#000000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5, 
                            popup=u"%s %s %s" % (row.Colegio, row['Promedio PSU'], row['Comuna'])))
map.save('colegios.html')
map

In [10]:
df.head()

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
0,1,Particular,Providencia,Cambridge College,29,704.4,-33.429730,-70.617110
1,2,Particular,Las Condes,Cordillera,63,699.0,-33.397130,-70.514321
2,3,Particular,Concón,Montemar,22,694.0,-32.951048,-71.535463
3,4,Particular,Maipú,Internacional Alba,26,693.4,-33.480574,-70.744990
4,5,Particular,Vitacura,Los Andes,69,692.9,-33.375795,-70.527009


In [11]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Toner')
i=0
for index, row in df.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#ff0000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5*row['Alumnos que rindieron la prueba']/100.0, 
                            popup=u"%s %s %s %s alumnos" % (row.Colegio, row['Promedio PSU'], row['Comuna'], row['Alumnos que rindieron la prueba'])))
map.save('colegios.html')
map

In [12]:
df.tail(20)

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
74,75,Particular,Ñuñoa,Colegio Suizo de Santiago,32,650.6,-33.456659,-70.608401
75,76,Particular,Peñalolén,Colegio Pumahue,88,650.1,-33.497000,-70.546760
77,78,Particular,Peñalolén,Colegio Pedro de Valdivia - Peñalolén,157,649.6,-33.493105,-70.576597
79,80,Particular,Las Condes,Colegio Wenlock School,49,649.0,-33.410778,-70.522065
80,81,Particular,Viña del Mar,Colegio Alemán de Valparaíso,65,648.3,-33.039007,-71.526086
82,83,Subvencionado,Vitacura,Liceo María Luisa Bombal,35,648.0,-33.388298,-70.579639
83,84,Particular,Lo Barnechea,Colegio Craighouse,106,647.7,-33.363000,-70.510160
84,85,Particular,Talca,Colegio Inglés,54,647.6,-35.450989,-71.596914
85,86,Particular,Constitución,Colegio Constitución,13,647.6,-35.329400,-72.413134
86,87,Subvencionado,La Serena,Colegio Gerónimo Rendic,112,647.5,-29.904190,-71.243704


In [13]:
df3 = df[df['Tipo de establecimiento'] != 'Particular']
df3

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
19,20,Subvencionado,Providencia,Josefino Santísima Trinidad,27,673.5,-33.435876,-70.618879
33,34,Municipal,Ñuñoa,Liceo Augusto D´Halmar,60,665.0,-33.460163,-70.581265
64,65,Subvencionado,San Vicente,Colegio El Salvador,65,652.5,-34.439299,-71.084709
82,83,Subvencionado,Vitacura,Liceo María Luisa Bombal,35,648.0,-33.388298,-70.579639
86,87,Subvencionado,La Serena,Colegio Gerónimo Rendic,112,647.5,-29.904190,-71.243704


In [14]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='OpenStreetMap')
colors = {
    'Particular': ('green', ''),
    'Subvencionado': ('red', 'star'),
    'Municipal': ('red', 'star-empty'),
}
for index, row in df.iterrows():
    color, icon = colors[row['Tipo de establecimiento']]
    folium.Marker(
        location=[row.lat,row.lng],
        popup=u"%s - %s - %s - %s" % (row['Tipo de establecimiento'], row.Colegio, row['Promedio PSU'], row['Comuna']),
        icon=folium.Icon(color=color, icon=icon)
    ).add_to(map)
map.save('colegios.html')
map

[mapa](colegios.html)